In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
import pickle

In [2]:
data = pd.read_csv('Churn_Modelling.csv')

In [3]:
## droping unimportant columns
data = data.drop(['RowNumber', 'CustomerId','Surname'], axis=1)

In [4]:
# Lets encode the gender first
label_encoder_gender = LabelEncoder()
data['Gender']= label_encoder_gender.fit_transform(data['Gender'])
data['Gender']

0       0
1       0
2       0
3       0
4       0
       ..
9995    1
9996    1
9997    0
9998    1
9999    0
Name: Gender, Length: 10000, dtype: int32

In [5]:
from sklearn.preprocessing import OneHotEncoder
onehot_encoder_geo=OneHotEncoder()
geo_encoded=onehot_encoder_geo.fit_transform(data[['Geography']]).toarray()
geo_encoded_df=pd.DataFrame(geo_encoded,columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [6]:
data = pd.concat([data.drop('Geography', axis=1), geo_encoded_df], axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [7]:
# Splitting data
X= data.drop('EstimatedSalary', axis=1)
y=data['EstimatedSalary']

In [20]:
X

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,1,39,5,0.00,2,1,0,0,1.0,0.0,0.0
9996,516,1,35,10,57369.61,1,1,1,0,1.0,0.0,0.0
9997,709,0,36,7,0.00,1,0,1,1,1.0,0.0,0.0
9998,772,1,42,3,75075.31,2,1,0,1,0.0,1.0,0.0


In [8]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)
##Scale down this feature with standard scalar
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.fit_transform(X_test)

In [9]:
## Save the encoders and scaler
with open('reg_label_encoder_gender.pkl','wb') as file:
    pickle.dump(label_encoder_gender,file)

with open('reg_onehot_encoder_geo.pkl', 'wb') as file:
    pickle.dump(onehot_encoder_geo,file)

with open('reg_scaler.pkl','wb') as file:
    pickle.dump(scaler,file)

Training ANN Regression

In [10]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [11]:
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1) # Default activation is linear activation (Regression)
])

m:\Learning_Workspace\AI\AI_Projects\KrishNaikProjects\ANN-DL-Algorithm\venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [13]:
model.compile(optimizer='adam', loss='mean_absolute_error', metrics=['mae'])
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [14]:
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime
log_dir="reg_logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callsback=TensorBoard(log_dir=log_dir,histogram_freq=1)

In [15]:
## Set early stopping - if model stops learning apply early stopping. If loss is not decreasing after certain epoch ---stop further training
early_stopping_callback=EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)

In [16]:
## Train the model
history=model.fit(
    X_train, y_train,validation_data=(X_test,y_test), epochs=100,
    callbacks=[tensorflow_callsback,early_stopping_callback]
)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 14s 31ms/step - loss: 99575.2344 - mae: 99575.2344 - val_loss: 98556.9609 - val_mae: 98556.9609
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 4s 15ms/step - loss: 99981.1016 - mae: 99981.1016 - val_loss: 97275.8828 - val_mae: 97275.8828
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 4s 16ms/step - loss: 98530.5938 - mae: 98530.5938 - val_loss: 93960.1250 - val_mae: 93960.1250
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 8s 31ms/step - loss: 94704.3125 - mae: 94704.3125 - val_loss: 88344.3828 - val_mae: 88344.3828
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 5s 22ms/step - loss: 88030.1328 - mae: 88030.1328 - val_loss: 80854.0938 - val_mae: 80854.0938
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 6s 24ms/step - loss: 80284.7031 - mae: 80284.7031 - val_loss: 72655.6250 - val_mae: 72655.6250
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 4s 17ms/step - loss: 72929.7812 - mae: 72929.7812 - val_loss: 64918.1211 - val_mae: 64918.1211
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 8s 31m

In [25]:
%load_ext tensorboard

In [27]:
%tensorboard --logdir reg_logs/fit

Reusing TensorBoard on port 6006 (pid 29632), started 0:00:27 ago. (Use '!kill 29632' to kill it.)

In [17]:
# Evaluate model on Test data
test_loss,test_mae=model.evaluate(X_test,y_test)
print(f'TEST MAE :{test_mae}')

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 51105.2773 - mae: 51105.2773
TEST MAE :50299.35546875


In [18]:
model.save('reg_model.h5')